In [ ]:
!pip install pydub
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from google.colab import files

# raw_audio.zip, jsons.zip 업로드
uploaded = files.upload()

Saving jsons.zip to jsons.zip
Saving raw_audio.zip to raw_audio.zip


In [ ]:
import os

os.makedirs("raw_audio", exist_ok=True)
os.makedirs("jsons", exist_ok=True)

# 업로드한 파일 압축 해제
!unzip -o raw_audio.zip -d raw_audio
!unzip -o jsons.zip -d jsons

print("압축 해제 완료")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5.화재_8737_label.wav:  mismatching "local" filename (5.эЩФьЮм_8737_label.wav),
         continuing with "central" filename version
  inflating: raw_audio/5.화재_8737_label.wav  
5.화재_8744_label.wav:  mismatching "local" filename (5.эЩФьЮм_8744_label.wav),
         continuing with "central" filename version
  inflating: raw_audio/5.화재_8744_label.wav  
5.화재_8746_label.wav:  mismatching "local" filename (5.эЩФьЮм_8746_label.wav),
         continuing with "central" filename version
  inflating: raw_audio/5.화재_8746_label.wav  
5.화재_8748_label.wav:  mismatching "local" filename (5.эЩФьЮм_8748_label.wav),
         continuing with "central" filename version
  inflating: raw_audio/5.화재_8748_label.wav  
5.화재_8751_label.wav:  mismatching "local" filename (5.эЩФьЮм_8751_label.wav),
         continuing with "central" filename version
  inflating: raw_audio/5.화재_8751_label.wav  
5.화재_8762_label.wav:  mismatching "local" filename (5.эЩФьЮм_8762_label.wav),
         co

In [ ]:
import json
import csv
from pydub import AudioSegment
import os

os.makedirs("trimmed", exist_ok=True)
csv_data = []

for json_file in sorted(os.listdir("jsons"))[:1500]:
    if not json_file.endswith(".json"):
        continue

    try:
        with open(os.path.join("jsons", json_file), "r", encoding="utf-8") as f:
            data = json.load(f)

        base_name = data["audio"]["fileName"] + "_label.wav"
        audio_path = os.path.join("raw_audio", base_name)
        if not os.path.exists(audio_path): continue

        ann = data["annotations"][0]
        start_ms = float(ann["area"]["start"]) * 1000
        end_ms = float(ann["area"]["end"]) * 1000
        text = ann["note"]

        audio = AudioSegment.from_wav(audio_path)
        segment = audio[start_ms:end_ms]
        segment = segment.set_frame_rate(16000).set_channels(1)

        out_path = os.path.join("trimmed", base_name)
        segment.export(out_path, format="wav")

        csv_data.append({"path": out_path, "text": text})

    except Exception as e:
        print(f"❌ {json_file} 처리 중 에러: {e}")
        continue

# train.csv 저장
with open("train.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["path", "text"])
    writer.writeheader()
    writer.writerows(csv_data)

print("✅ 1500개 전처리 완료")


✅ 1500개 전처리 완료
